# Convolutional Neural Networks

With the basic principles of multi-layer perceptrons understood, extending the same ideas to convolution relies on understanding that to a convolutional filter, a single sample can actually be multiple input samples that the discrete optimizer has to work with.